# Separating participants into groups

Simple script that builds a sheet in an excel file.

## Usage

In a camp participant's spreadsheet, this script takes the paying and confirmed participants and separates them into groups by age.

Then, read the files in the `source` folder — the script gets the first `xlsx` it finds.

In [1]:
from pathlib import Path

file_folder = Path("./source")


def find_first_xlsx():
    for filename in file_folder.glob("*.xlsx"):
        if "xlsx" in str(filename):
            return Path(filename)

Turns the sheet into a dataframe and sets the phone as string (to avoid confusion with numbers and zeroes)

In [2]:
import pandas as pd

path = find_first_xlsx()

df = pd.read_excel(
    path,
    sheet_name="LISTA DE PAGANTES (FINANCEIRA)",
    header=9,
)

Turns `Desistente` and `Pago` into booleans

In [3]:
df["Desistente"] = df["Desistente"] != "NÃO"
df["Pago?"] = df["Pago?"] == "PAGANTE"

Filters out team members — leaving only participants

In [4]:
df = df[df["Será Acampante ou Equipe?"] == "Sou participante (acampante)"]

Filters out non-paying and dropouts

In [5]:
df = df[df["Pago?"]]
df = df[~df["Desistente"]]

Remove useless columns

In [6]:
df.drop(
    columns=[
        'Telefone para contato (Ex: 11XXXXXXXXX sem traço "-" e sem pontos ".")',
        "Pago?",
        "Desistente",
    ],
    inplace=True,
)

Order by age and gender

In [7]:
df.sort_values(["Idade"], ascending=[True], inplace=True)

Create new column and add group using **round-robyn**

In [8]:
df["Grupo"] = 0

for i, index in enumerate(df.index):
    df.loc[index, "Grupo"] = (i % 6) + 1

Prepare new filename

In [9]:
import re

pattern = r"(?<=/)(.*?)(?=\.xlsx)"
match = re.search(pattern, str(path))

if not match:
    raise Exception('Wait, is there an xlsx file inside "source" folder?')

old_name = match.group(0)
new_path = re.sub(pattern, f"{old_name} - COM GRUPOS", str(path))

Write new excel file

In [10]:
import openpyxl

sheet_name = "Grupos"


workbook = openpyxl.load_workbook(path)

if sheet_name in workbook.sheetnames:
    sheet = workbook[sheet_name]
else:
    sheet = workbook.create_sheet(sheet_name)

# Clear sheet
sheet.delete_rows(1, sheet.max_row)  # Delete all rows if the sheet exists

row_index = 1

for c, col_name in enumerate(df.columns):  # Write the header
    cell = sheet.cell(row=1, column=c + 1)
    cell.value = col_name

for r in range(df.shape[0]):  # Iterate over rows
    for c in range(df.shape[1]):  # Iterate over columns
        cell = sheet.cell(
            row=r + 2, column=c + 1
        )  # +2 to start from the second row (after the header)
        cell.value = df.iloc[r, c]  # Write the value


workbook.save(new_path)
print(f"DataFrames written to {new_path} successfully.")

DataFrames written to source/Acampa  2025 - VOCARE (CENTRAL) - COM GRUPOS.xlsx successfully.
